<a href="https://colab.research.google.com/github/bilalProgTech/online-data-science-ml-challenges/blob/master/Machine-Hack-Soccer-Fever-ML-Challenge/Ensemble-Stacking-Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/soccer_fever_weekend_hackathon_edition_2_the_last_hacker_standing/LHS_WE_2_Soccer_Fever.zip
!unzip *.zip

--2021-08-22 18:54:56--  https://machinehack-be.s3.amazonaws.com/soccer_fever_weekend_hackathon_edition_2_the_last_hacker_standing/LHS_WE_2_Soccer_Fever.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.62.48
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.62.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261716 (256K) [application/octet-stream]
Saving to: ‘LHS_WE_2_Soccer_Fever.zip’

LHS_WE_2_Soccer_Fev 100%[===================>] 255.58K   294KB/s    in 0.9s    

2021-08-22 18:54:58 (294 KB/s) - ‘LHS_WE_2_Soccer_Fever.zip’ saved [261716/261716]

Archive:  LHS_WE_2_Soccer_Fever.zip
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

,season,date,league_id,league,Team 1,Team2,SPI1,SPI2,proj_score1,proj_score2,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,Outcome
0,2019,01/03/19,1979,Chinese Super League,Shandong Luneng,Guizhou Renhe,48.22,37.83,1.75,0.84,45.9,22.1,1.0,0.0,1.39,0.26,2.05,0.54,1.05,0.00,1
1,2019,01/03/19,1979,Chinese Super League,Shanghai Greenland,Shanghai SIPG,39.81,60.08,1.22,1.89,25.6,63.4,0.0,4.0,0.57,2.76,0.80,1.50,0.00,3.26,0
2,2019,01/03/19,1979,Chinese Super League,Guangzhou Evergrande,Tianjin Quanujian,65.59,39.99,2.58,0.62,77.1,28.8,3.0,0.0,0.49,0.45,1.05,0.75,3.15,0.00,1
3,2019,01/03/19,1979,Chinese Super League,Wuhan Zall,Beijing Guoan,32.25,54.82,1.10,1.79,35.8,58.9,0.0,1.0,1.12,0.97,1.51,0.94,0.00,1.05,0
4,2019,01/03/19,1979,Chinese Super League,Chongqing Lifan,Guangzhou RF,38.24,40.45,1.57,1.24,26.2,21.3,2.0,2.0,2.77,3.17,1.05,2.08,2.10,2.10,1


In [ ]:
train.columns

Index(['season', 'date', 'league_id', 'league', 'Team 1', 'Team2', 'SPI1',
       'SPI2', 'proj_score1', 'proj_score2', 'importance1', 'importance2',
       'score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1',
       'adj_score2', 'Outcome'],
      dtype='object')

In [ ]:
X = train[['SPI1', 'SPI2', 'proj_score1', 'proj_score2']]
y = train['Outcome']
X_test = test[['SPI1', 'SPI2', 'proj_score1', 'proj_score2']]

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
err = []
y_pred_tot_lgm = []

from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=15, shuffle=True, random_state=2020)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMClassifier(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       num_leaves=30,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='log_loss',
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print("err_lgm: ",log_loss(y_val,pred_y))
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 200 rounds.
[200]	training's binary_logloss: 0.0157516	valid_1's binary_logloss: 0.0284853
[400]	training's binary_logloss: 0.00572125	valid_1's binary_logloss: 0.0263127
[600]	training's binary_logloss: 0.00230068	valid_1's binary_logloss: 0.0245877
Early stopping, best iteration is:
[577]	training's binary_logloss: 0.00249898	valid_1's binary_logloss: 0.0239301
err_lgm:  0.023930120902250933
Training until validation scores don't improve for 200 rounds.
[200]	training's binary_logloss: 0.0160866	valid_1's binary_logloss: 0.0220627
[400]	training's binary_logloss: 0.00613895	valid_1's binary_logloss: 0.0154738
[600]	training's binary_logloss: 0.00245323	valid_1's binary_logloss: 0.0110912
[800]	training's binary_logloss: 0.00102352	valid_1's binary_logloss: 0.00974241
[1000]	training's binary_logloss: 0.000415507	valid_1's binary_logloss: 0.00917502
Early stopping, best iteration is:
[945]	training's binary_logloss: 0.000529055	valid_

In [ ]:
from xgboost import XGBClassifier

errxgb = []
y_pred_tot_xgb = []

from sklearn.model_selection import KFold,StratifiedKFold

fold = StratifiedKFold(n_splits=15, shuffle=True, random_state=2020)
i = 1
for train_index, test_index in fold.split(X,y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = XGBClassifier(boosting_type='gbdt',
                      max_depth=5,
                      learning_rate=0.07,
                      n_estimators=5000,
                      random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='logloss',
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print("err_xgb: ",log_loss(y_val,pred_y))
    errxgb.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_xgb.append(pred_test)

[0]	validation_0-logloss:0.629027	validation_1-logloss:0.629382
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 200 rounds.
[200]	validation_0-logloss:0.013855	validation_1-logloss:0.015945
[400]	validation_0-logloss:0.01179	validation_1-logloss:0.014828
[600]	validation_0-logloss:0.008668	validation_1-logloss:0.013389
[800]	validation_0-logloss:0.007251	validation_1-logloss:0.012798
[1000]	validation_0-logloss:0.006647	validation_1-logloss:0.013098
Stopping. Best iteration:
[806]	validation_0-logloss:0.007232	validation_1-logloss:0.01276

err_xgb:  0.012759913796040018
[0]	validation_0-logloss:0.629008	validation_1-logloss:0.629363
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 200 rounds.
[200]	validation_0-logloss:0.01403	validation_1-logloss:0.010631
[400]	validation_0-log

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 28 kB/s 


In [ ]:
from catboost import CatBoostClassifier,Pool, cv
errCB = []
y_pred_tot_cb = []
from sklearn.model_selection import KFold,StratifiedKFold

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
i = 1
for train_index, test_index in fold.split(X,y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = CatBoostClassifier(n_estimators=1000,
                           random_state=1994,
                           eval_metric='Logloss',
                           learning_rate=0.5, 
                           max_depth=10)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print("err_cb: ",log_loss(y_val,pred_y))
    errCB.append(log_loss(y_val,pred_y))
    pred_test = m.predict_proba(X_test)
    i = i + 1
    y_pred_tot_cb.append(pred_test)

0:	learn: 0.1448324	test: 0.1448324	test1: 0.1503370	best: 0.1503370 (0)	total: 72.8ms	remaining: 1m 12s
200:	learn: 0.0059744	test: 0.0059744	test1: 0.0133687	best: 0.0126063 (34)	total: 1.95s	remaining: 7.74s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.01260625685
bestIteration = 34

Shrink model to first 35 iterations.
err_cb:  0.012606256846356748
0:	learn: 0.1524155	test: 0.1524155	test1: 0.1578106	best: 0.1578106 (0)	total: 13.2ms	remaining: 13.2s
200:	learn: 0.0076644	test: 0.0076644	test1: 0.0319209	best: 0.0309791 (27)	total: 1.79s	remaining: 7.12s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.03097914024
bestIteration = 27

Shrink model to first 28 iterations.
err_cb:  0.030979140239778217
0:	learn: 0.1339663	test: 0.1339663	test1: 0.1401511	best: 0.1401511 (0)	total: 13.1ms	remaining: 13.1s
200:	learn: 0.0058079	test: 0.0058079	test1: 0.0334490	best: 0.0325097 (42)	total: 1.81s	remaining: 7.21s
Stopped by overfitting detector  

In [ ]:
(np.mean(errxgb, 0) + np.mean(err, 0) + np.mean(errCB, 0))/3

0.02990065580113462

In [ ]:
submission = pd.DataFrame((np.mean(y_pred_tot_lgm, 0)+np.mean(y_pred_tot_xgb, 0)+np.mean(y_pred_tot_cb, 0))/3)
submission.to_csv('LGB-XG-CB.csv', index=False)

In [ ]:
submission.head()

,0,1
0,0.000000e+00,1.000000e+00
1,1.192093e-07,9.999999e-01
2,9.999999e-01,3.578051e-08
3,0.000000e+00,1.000000e+00
4,1.000000e+00,7.638253e-11
